# CHALLENGE COLLECTING DATA - IMMOVLAN
## SECOND PART: CLEANING THE DATAFRAME

Due to the slow processing of the main.py module to scrape properties data, I run this notebook to clean the data results.

### DATA COLLECTION

In [1]:
# Set the notebook to show all outputs in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import glob
import os

# Pandas options for data wrangling and output set-up 
import pandas as pd
pd.set_option('display.max_columns', None) # display all columns
pd.set_option('display.expand_frame_repr', False) # print all columns and in the same line
pd.set_option('display.max_colwidth', None) # display the full content of each cell
pd.set_option('display.float_format', lambda x: '%.3f' %x) # floats to be displayed with 3 decimal places

In [2]:
# Concatenate all my batches (CSVs output)
path = "output"
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "immovlan_properties*.csv"))), ignore_index= True)

### A LOOK TO THE RESULTS

In [3]:
df.shape

(35325, 122)

As the target variable is Price, deleting records with missing price

In [4]:
df = df.dropna(subset = ['Prix'])

Also deleting duplicates based on Ref column

In [5]:
df = df.drop_duplicates(subset=["Ref"])

There are 121 columns.  
My scraping is taking all tags in the html structure of each page.  
While working in the scraping, I noticed some of the tags were hardly used in the website.  
--> remove all columns with over 50% NAs

In [6]:
# percentage of NA per column
naPct = df.isnull().sum()/df.shape[0]*100

# list of columns over 50% NAs
col_to_drop = naPct[naPct>50].keys()
df = df.drop(col_to_drop, axis=1)

# dropping as well any records that may have all NAs
df = df.dropna(how='all')


In [7]:
df.shape

(16686, 26)

This looks much better...

### TRANSFORMING THE DATA

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16686 entries, 0 to 35311
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ad_link                               16686 non-null  object 
 1   Ref                                   16686 non-null  object 
 2   Titre                                 16686 non-null  object 
 3   Prix                                  16686 non-null  float64
 4   Addresse                              13419 non-null  object 
 5   Localite                              16678 non-null  object 
 6   Etat_du_bien                          12724 non-null  object 
 7   Année_de_construction                 10276 non-null  float64
 8   Nombre_de_chambres_à_coucher          16340 non-null  float64
 9   Surface_habitable                     15497 non-null  object 
 10  Meublé                                9350 non-null   object 
 11  Caves               

In [9]:
# Differentiate numerical and categorical cols
numeric_cols = df.select_dtypes(include=np.number).columns
numeric_cols

categoric_cols = df.select_dtypes(exclude=np.number).columns
categoric_cols

Index(['Prix', 'Année_de_construction', 'Nombre_de_chambres_à_coucher',
       'Nombre_de_salles_de_bain', 'Nombre_de_toilettes', 'Nombre_de_façades'],
      dtype='object')

Index(['ad_link', 'Ref', 'Titre', 'Addresse', 'Localite', 'Etat_du_bien',
       'Surface_habitable', 'Meublé', 'Caves', 'Type_de_chauffage',
       'Ascenseur', 'Raccordé_à_leau_courante',
       'Conso_spécifique_dénergie_primaire',
       'Attestation__Installation_électrique', 'Type_de_zone_inondable',
       'Zone_inondable_délimitée', 'Terrasse_aménagée', 'Grenier',
       'Surface_totale_du_terrain', 'Jardin'],
      dtype='object')

In [10]:
def MemOptimisation(df):
    """
    By default pandas assign data types that consume a lot of memory.
    Also category data type handles much better categorical variables than object
    Also numerical variable seem to be Integers
    """
    print(f"\nAmount of memory used by all attributes: {df.memory_usage(deep=True).sum()}\n")
    
    # Optimise memory usage
    for i in categoric_cols:
        df[i] = df[i].astype('category')
    for i in numeric_cols:
        df[i] = df[i].astype('Int32')

    print(df.info(memory_usage='deep'))
    print("\nAmount of memory used now by all attributes: ",df.memory_usage(deep=True).sum())

In [11]:
MemOptimisation(df)


Amount of memory used by all attributes: 21719011

<class 'pandas.core.frame.DataFrame'>
Index: 16686 entries, 0 to 35311
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ad_link                               16686 non-null  category
 1   Ref                                   16686 non-null  category
 2   Titre                                 16686 non-null  category
 3   Prix                                  16686 non-null  Int32   
 4   Addresse                              13419 non-null  category
 5   Localite                              16678 non-null  category
 6   Etat_du_bien                          12724 non-null  category
 7   Année_de_construction                 10276 non-null  Int32   
 8   Nombre_de_chambres_à_coucher          16340 non-null  Int32   
 9   Surface_habitable                     15497 non-null  category
 10  Meublé                 

Let's have a look at the variables

In [12]:
df[numeric_cols].head()

,Prix,Année_de_construction,Nombre_de_chambres_à_coucher,Nombre_de_salles_de_bain,Nombre_de_toilettes,Nombre_de_façades
0,299000,2017,1,1,1,2
1,1275000,2014,3,3,4,2
2,549000,1900,6,2,2,2
3,360500,2024,<NA>,<NA>,<NA>,<NA>
4,845000,1896,3,1,2,2


All numeric variables seems to be correct and relevant for the goal of the project

In [13]:
df[categoric_cols].head()

,ad_link,Ref,Titre,Addresse,Localite,Etat_du_bien,Surface_habitable,Meublé,Caves,Type_de_chauffage,Ascenseur,Raccordé_à_leau_courante,Conso_spécifique_dénergie_primaire,Attestation__Installation_électrique,Type_de_zone_inondable,Zone_inondable_délimitée,Terrasse_aménagée,Grenier,Surface_totale_du_terrain,Jardin
0,https://immovlan.be/fr/detail/duplex/a-vendre/1000/bruxelles/vbc89694,VBC89694,Duplex à vendre,Rue une Personne 8,1000 Bruxelles,Excellent,55 m²,Non,Oui,Gaz,Oui,Oui,119 kWh/m²/an,"oui, attestation conforme",Pas de risque d’inondation,Information non communiquée,NaN,NaN,NaN,NaN
1,https://immovlan.be/fr/detail/appartement/a-vendre/1000/bruxelles/vbc89261,VBC89261,Appartement à vendre,Quai des Péniches 69 27A,1000 Bruxelles,Excellent,219 m²,Oui,Oui,Gaz,Oui,NaN,NaN,NaN,Information non communiquée,Information non communiquée,Oui,NaN,NaN,NaN
2,https://immovlan.be/fr/detail/maison/a-vendre/1000/bruxelles/vbc87245,VBC87245,Maison à vendre,Rue des Fabriques 15,1000 Bruxelles,À rénover,365 m²,NaN,NaN,Gaz,Non,Oui,798 kWh/m²/an,"oui, attestation non conforme",Pas de risque d’inondation,Information non communiquée,Oui,Oui,136 m²,NaN
3,https://immovlan.be/fr/detail/penthouse/a-vendre/1000/bruxelles/rbt73168,RBT73168,Penthouse à vendre,Congresstraat 17,1000 Bruxelles,NaN,NaN,NaN,Non,NaN,NaN,NaN,NaN,"non, pas d'attestation",Information non communiquée,Information non communiquée,NaN,Non,NaN,NaN
4,https://immovlan.be/fr/detail/appartement/a-vendre/1000/bruxelles/vbc91915,VBC91915,Appartement à vendre,NaN,1000 Bruxelles,NaN,315 m²,Non,Non,Gaz,Oui,Oui,249 kWh/m²/an,pas d'application,Information non communiquée,Information non communiquée,Oui,Non,NaN,NaN


In [14]:
# Remove 'à vendre' from the values and rename the column to Type_du_bien
df['Titre'] = df['Titre'].str.replace('à vendre', '', regex=False).str.strip()
df = df.rename(columns={'Titre': 'Type_du_bien'})

In [15]:
# Split column localite into zip and localite
df['Zip'] = df['ad_link'].str.extract(r'/a-vendre/(\d{4})/')[0]
df['Localite'] = df['ad_link'].str.extract(r'/a-vendre/\d{4}/([^/]+)/')[0].str.capitalize()

Confirm that below columns are binary (Oui/Non) and then transform them

In [16]:
cols = ['Meublé', 'Caves', 'Ascenseur', 'Raccordé_à_leau_courante','Terrasse_aménagée', 'Grenier', 'Jardin']
for c in cols:
    df[c].value_counts()

Meublé
Non    8875
Oui     475
Name: count, dtype: int64

Caves
Non    5459
Oui    4100
Name: count, dtype: int64

Ascenseur
Non    8380
Oui    3653
Name: count, dtype: int64

Raccordé_à_leau_courante
Oui    8490
Non      27
Name: count, dtype: int64

Terrasse_aménagée
Oui    11748
Non      585
Name: count, dtype: int64

Grenier
Oui    5281
Non    4074
Name: count, dtype: int64

Jardin
Oui    9434
Non     861
Name: count, dtype: int64

In [17]:
for c in cols:
    df[c] = df[c].map({'Oui': 1, 'Non': 0})

### FINAL DATAFRAME

In [21]:
df.head()

,ad_link,Ref,Type_du_bien,Prix,Addresse,Localite,Etat_du_bien,Année_de_construction,Nombre_de_chambres_à_coucher,Surface_habitable,Meublé,Caves,Nombre_de_salles_de_bain,Nombre_de_toilettes,Type_de_chauffage,Ascenseur,Nombre_de_façades,Raccordé_à_leau_courante,Conso_spécifique_dénergie_primaire,Attestation__Installation_électrique,Type_de_zone_inondable,Zone_inondable_délimitée,Terrasse_aménagée,Grenier,Surface_totale_du_terrain,Jardin,Zip
0,https://immovlan.be/fr/detail/duplex/a-vendre/1000/bruxelles/vbc89694,VBC89694,Duplex,299000,Rue une Personne 8,Bruxelles,Excellent,2017,1,55 m²,0,1,1,1,Gaz,1,2,1,119 kWh/m²/an,"oui, attestation conforme",Pas de risque d’inondation,Information non communiquée,NaN,NaN,NaN,NaN,1000
1,https://immovlan.be/fr/detail/appartement/a-vendre/1000/bruxelles/vbc89261,VBC89261,Appartement,1275000,Quai des Péniches 69 27A,Bruxelles,Excellent,2014,3,219 m²,1,1,3,4,Gaz,1,2,NaN,NaN,NaN,Information non communiquée,Information non communiquée,1,NaN,NaN,NaN,1000
2,https://immovlan.be/fr/detail/maison/a-vendre/1000/bruxelles/vbc87245,VBC87245,Maison,549000,Rue des Fabriques 15,Bruxelles,À rénover,1900,6,365 m²,NaN,NaN,2,2,Gaz,0,2,1,798 kWh/m²/an,"oui, attestation non conforme",Pas de risque d’inondation,Information non communiquée,1,1,136 m²,NaN,1000
3,https://immovlan.be/fr/detail/penthouse/a-vendre/1000/bruxelles/rbt73168,RBT73168,Penthouse,360500,Congresstraat 17,Bruxelles,NaN,2024,<NA>,NaN,NaN,0,<NA>,<NA>,NaN,NaN,<NA>,NaN,NaN,"non, pas d'attestation",Information non communiquée,Information non communiquée,NaN,0,NaN,NaN,1000
4,https://immovlan.be/fr/detail/appartement/a-vendre/1000/bruxelles/vbc91915,VBC91915,Appartement,845000,NaN,Bruxelles,NaN,1896,3,315 m²,0,0,1,2,Gaz,1,2,1,249 kWh/m²/an,pas d'application,Information non communiquée,Information non communiquée,1,0,NaN,NaN,1000


In [22]:
df.shape

(16686, 27)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16686 entries, 0 to 35311
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ad_link                               16686 non-null  category
 1   Ref                                   16686 non-null  category
 2   Type_du_bien                          16686 non-null  object  
 3   Prix                                  16686 non-null  Int32   
 4   Addresse                              13419 non-null  category
 5   Localite                              16686 non-null  object  
 6   Etat_du_bien                          12724 non-null  category
 7   Année_de_construction                 10276 non-null  Int32   
 8   Nombre_de_chambres_à_coucher          16340 non-null  Int32   
 9   Surface_habitable                     15497 non-null  category
 10  Meublé                                9350 non-null   category
 11  Caves  

The final dataframe shows:
- 16686 records
- 27 variables

In [25]:
# Saving the dataframe as immovlan_properties_FINAL.csv
file_name = f"immovlan_properties_FINAL.csv"
df.to_csv(file_name, index=False, encoding="utf-8-sig")